In [1]:
using System.IO;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Linq.Expressions;

var lineSeparator = "\r\n";
var ModelNumber = "9000";

var inputFunction = (string day, bool isTest) => {
    var file = File.ReadAllText($"InputFiles\\D{day}{(isTest ? "_test" : "")}.txt");
    lineSeparator = file.Contains("\r\n") ? "\r\n" : "\n";
    return file.Split(lineSeparator + lineSeparator, StringSplitOptions.RemoveEmptyEntries);
};

var CrateMoverFunc = (Stack<string> fromStack, Stack<string> toStack, int cratesToMove) => {
    switch(ModelNumber){
        case "9000":
            while(cratesToMove > 0){
                toStack.Push(fromStack.Pop());
                cratesToMove--;
            }
        break;
        case "9001":
            var stagingStack = new Stack<string>();
            while(cratesToMove > 0){
                stagingStack.Push(fromStack.Pop());
                cratesToMove--;
            }
            string crate;
            while(stagingStack.TryPop(out crate)){
                toStack.Push(crate);
            }
        break;
    }
};

var solvingFunction = (string[] crateLines, string[] procs) => {
    // load stacks
    var crateStacks = new Dictionary<int, Stack<string>>();
    var stacks = crateLines[crateLines.Length - 1].Where(c => c != ' ').Select(c => new string(c, 1));
    
    foreach(var stack in stacks)
    {
        var stackNo = Convert.ToInt32(stack);
        var stackPos = crateLines[crateLines.Length - 1].IndexOf(stack);
        
        crateStacks.Add(stackNo, new Stack<string>());
        foreach(var crateLine in crateLines.Reverse().Skip(1))//ignore last entry when loading data - not needed
        {
            var crate = crateLine.Substring(stackPos, 1);
            if (!String.IsNullOrWhiteSpace(crate)){
                crateStacks[stackNo].Push(crate);
            }
        }
    }
    
    // apply procs
    foreach(var proc in procs){
        // parse proc
        var moveCount = Convert.ToInt32(proc.Substring(4, proc.IndexOf("from") - 4).Trim());
        var movingStacks = new string(proc.Skip(proc.LastIndexOf("from") + 5).ToArray()).Split(" to ");
        var fromStackNo = Convert.ToInt32(movingStacks[0]);
        var toStackNo = Convert.ToInt32(movingStacks[1]);
        CrateMoverFunc(crateStacks[fromStackNo], crateStacks[toStackNo], moveCount);
    }
    var sb = new StringBuilder();
    foreach(var crateStack in crateStacks){
        sb.Append(crateStack.Value.Pop());
    }
    return sb.ToString();
};

var puzzleFunction = (bool isTest, Func<string, bool, string[]> inputFunction, Func<string[], string[], string> solvingFunction) => {
    var input = inputFunction("05", isTest);
    var crateLines = input[0].Split(lineSeparator);
    var procs = input[1].Split(lineSeparator, StringSplitOptions.RemoveEmptyEntries);
    var topStack = solvingFunction(crateLines, procs);
    Console.WriteLine((isTest ? "Test" : "Puzzle") + $" Result: {topStack}");
};

Console.WriteLine("Problem 01:");
puzzleFunction(true, inputFunction, solvingFunction);
puzzleFunction(false, inputFunction, solvingFunction);

Console.WriteLine("");
Console.WriteLine("Problem 02:");
ModelNumber = "9001";
puzzleFunction(true, inputFunction, solvingFunction);
puzzleFunction(false, inputFunction, solvingFunction);




Problem 01:
Test Result: CMZ
Puzzle Result: MQSHJMWNH

Problem 02:
Test Result: MCD
Puzzle Result: LLWJRBHVZ
